# This notebook is still under construction.

## Evaluation

Batch prediction and evaluation are very similar. They are based on DataFlow pipeline and CloudML provides Evaluate and Prediction DataFlow transform. Datalab can generate DataFlow pipeline code template for you, just like Preprocessing.

Run "%ml evaluate" to generate input cell. After fill in the required fields, run it to generate code.

In [ ]:
%%ml evaluate
preprocessed_eval_data_path: /content/datalab/tmp/ml/census/preprocessed/features_eval
metadata_path: /content/datalab/tmp/ml/census/preprocessed/metadata.yaml
model_dir: /content/datalab/tmp/ml/census/model/model
output_dir: /content/datalab/tmp/ml/census/evaluate

In [2]:

# header
"""
Following code is generated from command line:
%%ml evaluate
preprocessed_eval_data_path: /content/datalab/tmp/ml/census/preprocessed/features_eval
metadata_path: /content/datalab/tmp/ml/census/preprocessed/metadata.yaml
model_dir: /content/datalab/tmp/ml/census/model/model
output_dir: /content/datalab/tmp/ml/census/evaluate

Please modify as appropriate!!!
"""

# imports
import apache_beam as beam
from apache_beam.io import fileio
import google.cloud.ml as ml
import google.cloud.ml.analysis as analysis
import google.cloud.ml.dataflow.io.tfrecordio as tfrecordio
import google.cloud.ml.io as io
import json
import os

# defines
def extract_values((example, prediction)):
  import tensorflow as tf
  tf_example = tf.train.Example()
  tf_example.ParseFromString(example.values()[0])
  feature_map = tf_example.features.feature
  values = {'target': feature_map['target'].float_list.value[0]}
  values.update(prediction)
  return values

OUTPUT_DIR = '/content/datalab/tmp/ml/census/evaluate'
pipeline = beam.Pipeline('DirectPipelineRunner')


# evaluation
eval_parameters = tfrecordio.TFRecordParameters(
    file_path_prefix='/content/datalab/tmp/ml/census/preprocessed/features_eval',
    file_name_suffix='',
    shard_file=False,
    compress_file=True)
eval_features = pipeline | io.LoadFeatures('LoadEvalFeatures', eval_parameters)
trained_model = pipeline | io.LoadModel('LoadModel', '/content/datalab/tmp/ml/census/model/model')
evaluations = (eval_features | ml.Evaluate(trained_model, label='Evaluate')
    | beam.Map('ExtractEvaluationResults', extract_values))
eval_data_sink = beam.io.TextFileSink(os.path.join(OUTPUT_DIR, 'eval'))
evaluations | beam.Write('WriteEval', eval_data_sink)

# analysis

# run pipeline
pipeline.run()


View the output eval file:

In [3]:
!head /content/datalab/tmp/ml/census/evaluate/eval*

{u'score': [58987.35546875], 'target': 172300.0, u'key': ['705']}
{u'score': [45260.84375], 'target': 28000.0, u'key': ['1204']}
{u'score': [34258.0546875], 'target': 37300.0, u'key': ['1758']}
{u'score': [66288.8828125], 'target': 42400.0, u'key': ['2534']}
{u'score': [59149.06640625], 'target': 100000.0, u'key': ['3172']}
{u'score': [63032.22265625], 'target': 53000.0, u'key': ['3823']}
{u'score': [60508.41015625], 'target': 63300.0, u'key': ['4245']}
{u'score': [67393.390625], 'target': 56200.0, u'key': ['4918']}
{u'score': [47335.28515625], 'target': 132000.0, u'key': ['5352']}
{u'score': [41844.171875], 'target': 55000.0, u'key': ['6111']}


To generate a pipeline that runs in cloud, simply add --cloud to "%ml evaluate". Also all paths need to be GCS paths.

In [4]:
import os

bucket = 'gs://' + datalab_project_id() + '-sampledata'
eval_data_path = os.path.join(bucket, 'census', 'preprocessed', 'features_eval')
metadata_path = os.path.join(bucket, 'census', 'preprocessed', 'metadata.yaml')
model_path = os.path.join(bucket, 'census', 'trained', 'model')
output_dir = os.path.join(bucket, 'census', 'evaluate')
eval_file = os.path.join(output_dir, 'eval*')

In [ ]:
%%ml evaluate --cloud
preprocessed_eval_data_path: $eval_data_path
metadata_path: $metadata_path
model_dir: $model_path
output_dir: $output_dir

The generated code is like the following:

```

# header
"""
Following code is generated from command line:
%%ml evaluate --cloud
preprocessed_eval_data_path: $eval_data_path
metadata_path: $metadata_path
model_dir: $model_path
output_dir: $output_dir

Please modify as appropriate!!!
"""

# imports
import apache_beam as beam
from apache_beam.io import fileio
import google.cloud.ml as ml
import google.cloud.ml.analysis as analysis
import google.cloud.ml.dataflow.io.tfrecordio as tfrecordio
import google.cloud.ml.io as io
import json
import os

# defines
def extract_values((example, prediction)):
  import tensorflow as tf
  tf_example = tf.train.Example()
  tf_example.ParseFromString(example.values()[0])
  feature_map = tf_example.features.feature
  values = {'target': feature_map['target'].float_list.value[0]}
  values.update(prediction)
  return values

OUTPUT_DIR = 'gs://cloud-ml-test-automated-sampledata/census/evaluate'
import datetime
options = {
    'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
    'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
    'job_name': 'evaluate' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S'),
    'project': 'cloud-ml-test-automated',
    'extra_packages': ['gs://cloud-ml/sdk/cloudml-0.1.2.latest.tar.gz'],
    'teardown_policy': 'TEARDOWN_ALWAYS',
    'no_save_main_session': True
}
opts = beam.pipeline.PipelineOptions(flags=[], **options)
pipeline = beam.Pipeline('DataflowPipelineRunner', options=opts)


# evaluation
eval_parameters = tfrecordio.TFRecordParameters(
    file_path_prefix='gs://cloud-ml-test-automated-sampledata/census/preprocessed/features_eval',
    file_name_suffix='',
    shard_file=False,
    compress_file=True)
eval_features = pipeline | io.LoadFeatures('LoadEvalFeatures', eval_parameters)
trained_model = pipeline | io.LoadModel('LoadModel', 'gs://cloud-ml-test-automated-sampledata/census/trained/model')
evaluations = (eval_features | ml.Evaluate(trained_model, label='Evaluate')
    | beam.Map('ExtractEvaluationResults', extract_values))
eval_data_sink = beam.io.TextFileSink(os.path.join(OUTPUT_DIR, 'eval'))
evaluations | beam.Write('WriteEval', eval_data_sink)

# analysis

# run pipeline
pipeline.run()
```

After you run the above generated code, you can go to Developer Console to see the DataFlow job: https://pantheon.corp.google.com/dataflow (and select the right project). Also you can check the results as below:

In [6]:
!gsutil cat $eval_file | head -5

{u'score': [62152.2109375], 'target': 172300.0, u'key': ['705']}
{u'score': [50811.1875], 'target': 28000.0, u'key': ['1204']}
{u'score': [41454.01953125], 'target': 37300.0, u'key': ['1758']}
{u'score': [70818.65625], 'target': 42400.0, u'key': ['2534']}
{u'score': [63665.48046875], 'target': 100000.0, u'key': ['3172']}
